# 自定义输出解析器

在某些情况下，您可能希望实现自定义解析器以将模型输出构造为自定义格式。

有两种方法可以实现自定义解析器：
- 在 LCEL 中使用 RunnableLambda 或 RunnableGenerator – 我们强烈建议大多数用例使用此方法
- 通过从基类之一继承进行解析——这是困难方法

这两种方法之间的差异大多是表面的，主要在于触发哪些回调（例如， on_chain_start 与 on_parser_start ）

## 可运行的 Lambda 和生成器

推荐的解析方法是使用可运行的 lambda 和可运行的生成器！


### 使用RunnableLambda的方式

这个例子将创建一个解析器，用于将文本转换为大写并提取所有数字：

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import re
import os

load_dotenv()
api_key = os.getenv("ZHIPUAI_API_KEY")
base_url = os.getenv("ZHIPUAI_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0, model="glm-4-plus", max_tokens=8192)

prompt = ChatPromptTemplate.from_template(
    "给我讲一个包含数字的故事，长度在100字以内"
)

# 自定义解析函数
def custom_parser(message) -> dict:
    # 确保我们获取到文本内容
    text = message.content if hasattr(message, 'content') else str(message)
    # 提取所有的数字
    numbers = re.findall(r'\d+', text)
    # 统计文字的字数，包含标点符号
    word_count = len(text)
    return {
        "original_text": text,
        "numbers_found": numbers,
        "numbers_count": len(numbers),
        "word_count": word_count
    }

# 创建可运行的lambda
runnable = RunnableLambda(custom_parser)

# 将解析器添加到链中
chain = prompt | llm | runnable

result = chain.invoke({})
print(result)

{'original_text': '小明有3个苹果，他给了小红1个。后来，小华又给了小明5个。小明现在有7个苹果。一天，他吃掉了2个，还剩5个。第二天，他发现又多了4个，总数变成了9个。小明决定分给朋友们，每人3个，刚好分完。', 'numbers_found': ['3', '1', '5', '7', '2', '5', '4', '9', '3'], 'numbers_count': 9, 'word_count': 95}


### 使用RunnableGenerator的方式

这个例子将创建一个生成器解析器，用于逐步处理文本并生成分析结果

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableGenerator
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from typing import Iterator, Dict, Optional, Any, AsyncIterator
from langchain_core.runnables import RunnableConfig
import os
import asyncio



load_dotenv()
api_key = os.getenv("ZHIPUAI_API_KEY")
base_url = os.getenv("ZHIPUAI_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0, model="glm-4-plus", max_tokens=8192)

prompt = ChatPromptTemplate.from_template(
    "给我讲一个关于{topic}的主题文章，至少包含3个段落，字数300字以内。"
)

# 自定义生成器解析类
class ParagraphAnalyzer(RunnableGenerator):
    def __init__(self):
        self.name = "ParagraphAnalyzer" 
        self.paragraph_count = 0
    
    def invoke(self, input_text: Any,config: Optional[RunnableConfig] = None) -> Iterator[Dict]:
        # 确保我们获取到文本内容
        text = input_text.content if hasattr(input_text, 'content') else str(input_text)
        paragraphs = text.split('\n\n')
        
        for i, paragraph in enumerate(paragraphs, 1):
            if not paragraph.strip():
                continue
            
            # 分析每个段落
            word_count = len(paragraph.split())
            char_count = len(paragraph)
            
            yield {
                "paragraph_number": i,
                "content": paragraph,
                "word_count": word_count,
                "character_count": char_count,
                "analysis_complete": f"已完成第{i}段分析，这个段落有{word_count}个单词，{char_count}个字符。"
            }

        
parser = ParagraphAnalyzer()

# 将解析器添加到链中
chain = prompt | llm | parser
        

In [3]:
# 使用生成器方式获取结果，直接遍历生成, 同步调用
result = chain.invoke({"topic": "大模型应用开发"})
for item in result:
    print("-" * 50)
    print(f"段落 {item['paragraph_number']}:")
    print(f"内容: {item['content']}")
    print(f"字数: {item['word_count']}")
    print(f"字符数: {item['character_count']}")
    print(f"进度: {item['analysis_complete']}")
    print("-" * 50)

--------------------------------------------------
段落 1:
内容: **大模型应用开发：开启智能新时代**
字数: 1
字符数: 19
进度: 已完成第1段分析，这个段落有1个单词，19个字符。
--------------------------------------------------
--------------------------------------------------
段落 2:
内容: 随着人工智能技术的飞速发展，大模型应用开发成为推动行业创新的重要力量。大模型具备强大的数据处理和模式识别能力，能够广泛应用于自然语言处理、图像识别等领域，为企业提供高效的智能化解决方案。
字数: 1
字符数: 93
进度: 已完成第2段分析，这个段落有1个单词，93个字符。
--------------------------------------------------
--------------------------------------------------
段落 3:
内容: 在开发过程中，数据准备和模型训练是关键环节。高质量的数据集能够提升模型的准确性和泛化能力，而精细化的训练策略则能确保模型在特定场景下的优异表现。此外，开发者还需关注模型的优化和部署，以实现高效、稳定的运行。
字数: 1
字符数: 103
进度: 已完成第3段分析，这个段落有1个单词，103个字符。
--------------------------------------------------
--------------------------------------------------
段落 4:
内容: 大模型应用的开发不仅提升了业务效率，更开启了智能新时代的大门。未来，随着技术的不断进步，大模型将在更多领域发挥巨大潜力，助力各行各业实现智能化转型。
字数: 1
字符数: 74
进度: 已完成第4段分析，这个段落有1个单词，74个字符。
--------------------------------------------------
